In [1]:
!pip install scikit-learn
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from sklearn import model_selection, datasets, linear_model, metrics

### Ансамбль методов для классификации

### Стекинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingClassifier.html

#### Обработка датасета

# Классификация

In [2]:
credit_df = pd.read_csv('../data/Credit_risk_preprocessed.csv')


In [3]:
credit_df["default"].unique()
x = credit_df.iloc[:,:-1].values
y = credit_df.iloc[:,4].values
X_train_cls, X_test_cls, y_train_cls, y_test_cls = train_test_split(x, y, test_size = 0.3, shuffle=False)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train_cls)

X_train_cls = scaler.transform(X_train_cls)
X_test_cls = scaler.transform(X_test_cls)

### Стекинг без подбора гиперпараметров для него и базовых алгоритмов

In [4]:
estimators = [('lr', LogisticRegression()), ('dt', DecisionTreeClassifier())]
modelClf1 = StackingClassifier(estimators=estimators)

modelClf1.fit(X_train_cls, y_train_cls)
print('Стекинг для логичтической регрессии без гиперпараметров:',modelClf1.score(X_test_cls, y_test_cls))

Стекинг для логичтической регрессии без гиперпараметров: 0.9716666666666667


#### Подбор гиперпараметров для базового алгоритма дерева решений

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn import tree
parameters = {'max_depth':[1, 2, 3, 4, 5], 'min_samples_leaf': range (1, 8) ,'min_samples_split': range (2,10,2)}
dtc = tree.DecisionTreeClassifier()
grid1_ = GridSearchCV(dtc, parameters)

In [6]:
grid1_.fit(X_train_cls, y_train_cls)
print(grid1_.best_params_)
print(grid1_.best_estimator_)

{'max_depth': 5, 'min_samples_leaf': 3, 'min_samples_split': 2}
DecisionTreeClassifier(max_depth=5, min_samples_leaf=3)


In [7]:
tr1 = tree.DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 1, min_samples_split = 2)
tr1 = tr1.fit(X_train_cls, y_train_cls)
y_pred1 = tr1.predict(X_test_cls)
print('Стекинг для дерева решений с подбором гиперпараметров:',tr1.score(X_test_cls, y_test_cls))

Стекинг для дерева решений с подбором гиперпараметров: 0.9416666666666667


#### Подбор гиперпараметров для базового алгоритма логистической регрессии

In [8]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import tree
parameters = {'penalty': ('l1', 'l2', 'elasticnet', 'none'),'solver' : ['liblinear', 'sag', 'saga'], 'C': np.logspace(-4, 4, 20) }
lr = LogisticRegression()
grid2_ = GridSearchCV(lr, parameters)
grid2_.fit(X_train_cls, y_train_cls)

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                         'penalty': ('l1', 'l2', 'elasticnet', 'none'),
                         'solver': ['liblinear', 'sag', 'saga']})

In [9]:
print(grid2_.best_params_)
print(grid2_.best_estimator_)

{'C': 0.23357214690901212, 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=0.23357214690901212, solver='liblinear')


In [10]:
lr1 = LogisticRegression(C=1.623776739188721, penalty='l1', solver='liblinear')
lr1.fit(X_train_cls, y_train_cls)
y_pred = lr1.predict(X_test_cls)
print('Стекинг для логистической регресии с подбором гипер параметров:',lr1.score(X_test_cls, y_test_cls))

Стекинг для логистической регресии с подбором гипер параметров: 0.945


## Бэггинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

### Бэггинг без подбора гиперпараметров для него и базовых алгоритмов

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf3 = BaggingClassifier(base_estimator=LogisticRegression())

modelClf3.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Логистическая регрессия":',modelClf3.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Логистическая регрессия": 0.9466666666666667


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf4 = BaggingClassifier(base_estimator=DecisionTreeClassifier())

modelClf4.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Дерево решений":',modelClf4.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Дерево решений": 0.99


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
modelClf5 = BaggingClassifier(base_estimator=LogisticRegression(C=0.615848211066026, penalty='l1', solver='liblinear'))

modelClf5.fit(X_train_cls, y_train_cls)
print('Бэггинг с базовым алгоритмом "Логистическая регрессия с подбором гиперпараметров":',modelClf5.score(X_test_cls, y_test_cls))

Бэггинг с базовым алгоритмом "Логистическая регрессия с подбором гиперпараметров": 0.94


## Адаптивный бустинг

### Документация 
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier

### Адаптивный бустинг без подбора гиперпараметров для него и базовых алгоритмов

In [14]:
from sklearn.ensemble import AdaBoostClassifier

In [15]:
modelClf7 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
modelClf7.fit(X_train_cls, y_train_cls)
print('Адаптивный бустинг для дерева решений без гиперпараметров:',modelClf7.score(X_test_cls, y_test_cls))

Адаптивный бустинг для дерева решений без гиперпараметров: 0.9816666666666667


#### Подбор гиперпараметров для бустинга

In [16]:
parameters = {
              'n_estimators':[50,60,70],
              'algorithm': ['SAMME', 'SAMME.R'],
             }
abc1 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier())
grid5_ = GridSearchCV(abc1, parameters)

In [17]:
grid5_.fit(X_train_cls, y_train_cls)
print(grid5_.best_params_)
print(grid5_.best_estimator_)

{'algorithm': 'SAMME.R', 'n_estimators': 50}
AdaBoostClassifier(base_estimator=DecisionTreeClassifier())


In [18]:
abc2 = AdaBoostClassifier(algorithm='SAMME', base_estimator=DecisionTreeClassifier(),
                   n_estimators=70)
abc2.fit(X_train_cls, y_train_cls)
print('Адаптивный бустинг для дерева решений с гиперпараметрами', abc2.score(X_test_cls, y_test_cls))

Адаптивный бустинг для дерева решений с гиперпараметрами 0.9883333333333333


## Градиентный бустинг

### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

In [20]:
modelClf8 = GradientBoostingClassifier()
modelClf8.fit(X_train_cls, y_train_cls)
print(modelClf8.score(X_test_cls, y_test_cls))

0.99


#### Подбор гиперпараметоров для градиентного бустинга

In [21]:
parameters = {
              'n_estimators':[100,150,200,300],
              'max_depth':[2,3,4,5],
              'criterion':['friedman_mse', 'squared_error'],
             }
gbc1 = GradientBoostingClassifier()
grid6_ = GridSearchCV(gbc1, parameters)

In [22]:
grid6_.fit(X_train_cls, y_train_cls)
print(grid6_.best_params_)
print(grid6_.best_estimator_)

{'criterion': 'friedman_mse', 'max_depth': 2, 'n_estimators': 150}
GradientBoostingClassifier(max_depth=2, n_estimators=150)


In [23]:
gbc2 = GradientBoostingClassifier(max_depth=2,criterion='friedman_mse', n_estimators = 100)
gbc2.fit(X_train_cls, y_train_cls)
print(gbc2.score(X_test_cls, y_test_cls))

0.9916666666666667


### Ансамбль методов для регрессии

### Стекинг без подбора гиперпараметров для него и базовых алгоритмов

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html#sklearn.ensemble.StackingRegressor

In [24]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [25]:
student_df = pd.read_csv('../data/student_math_preprocessed.csv')

student_df["useAlc"].unique()
x = credit_df.iloc[:,:-1].values
y = credit_df.iloc[:,4].values
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(x, y,  test_size = 0.2, random_state=0)

In [26]:
estimators = [
    ('dt', DecisionTreeRegressor(random_state=42)),
     ('lr', linear_model.LinearRegression())]
modelClf10 = StackingRegressor(estimators=estimators)

modelClf10.fit(X_train_reg, y_train_reg)
print(modelClf10.score(X_test_reg, y_test_reg))

0.8171296739083087


### Бэггинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html#sklearn.ensemble.BaggingRegressor

In [27]:
from sklearn.ensemble import BaggingRegressor
modelClf12 = BaggingRegressor()
modelClf12.fit(X_train_reg, y_train_reg)
print(modelClf12.score(X_test_reg, y_test_reg))

0.9076352604078881


#### Подбор гиперпараметров для бэггинга

In [28]:
parameters = {
              'n_estimators':[2,3,4,5,6,8,9,10,20,15],
              'max_samples':[1.0,2.0,3.0],
              #'bootstrap':['True','False'],
              #'bootstrap_features':['True','False'],
              #'oob_score':['True','False']
             }
bgr2 = BaggingRegressor(base_estimator=LinearRegression())
grid8_ = GridSearchCV(bgr2, parameters)

In [29]:
grid8_.fit(X_train_reg, y_train_reg)
print(grid8_.best_params_)
print(grid8_.best_estimator_)

{'max_samples': 1.0, 'n_estimators': 4}
BaggingRegressor(base_estimator=LinearRegression(), n_estimators=4)


In [30]:
modelClf14 = BaggingRegressor(base_estimator = DecisionTreeRegressor(),max_samples=1.0, n_estimators=20)
modelClf14.fit(X_train_reg, y_train_reg)
print(modelClf14.score(X_test_reg, y_test_reg))

0.9047137479633688


### Бустинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html#sklearn.ensemble.AdaBoostRegressor

In [31]:
from sklearn.ensemble import AdaBoostRegressor
modelClf15 = AdaBoostRegressor()
modelClf15.fit(X_train_reg,y_train_reg)
print(modelClf15.score(X_test_reg, y_test_reg))

0.6955334484934264


#### Градиентный бустинг без подбора гиперпараметров

#### Документация
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html#sklearn.ensemble.GradientBoostingRegressor

In [32]:
from sklearn.ensemble import GradientBoostingRegressor
modelClf17 = GradientBoostingRegressor()
modelClf17.fit(X_train_reg, y_train_reg)
print(modelClf17.score(X_test_reg, y_test_reg))

0.8129090924328016


## Catboost

#### Открытый код 
https://github.com/catboost

In [33]:
!pip install catboost

In [34]:
def print_classification_model_metrics(estimator, y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print(estimator.score(X_test_cls, y_test))

from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import classification_report, confusion_matrix
cbc = CatBoostClassifier(learning_rate=0.15).fit(X_train_cls, y_train_cls)
print_classification_model_metrics(cbc, y_test_cls, cbc.predict(X_test_cls))

0:	learn: 0.4692953	total: 155ms	remaining: 2m 34s
1:	learn: 0.3302720	total: 160ms	remaining: 1m 19s
2:	learn: 0.2408309	total: 164ms	remaining: 54.5s
3:	learn: 0.1745150	total: 168ms	remaining: 41.8s
4:	learn: 0.1420410	total: 172ms	remaining: 34.2s
5:	learn: 0.1235028	total: 177ms	remaining: 29.3s
6:	learn: 0.1076271	total: 181ms	remaining: 25.6s
7:	learn: 0.0933984	total: 185ms	remaining: 22.9s
8:	learn: 0.0819392	total: 190ms	remaining: 20.9s
9:	learn: 0.0757649	total: 193ms	remaining: 19.2s
10:	learn: 0.0727758	total: 198ms	remaining: 17.8s
11:	learn: 0.0714588	total: 202ms	remaining: 16.6s
12:	learn: 0.0679745	total: 206ms	remaining: 15.7s
13:	learn: 0.0628354	total: 210ms	remaining: 14.8s
14:	learn: 0.0602450	total: 214ms	remaining: 14s
15:	learn: 0.0558751	total: 217ms	remaining: 13.4s
16:	learn: 0.0512265	total: 221ms	remaining: 12.8s
17:	learn: 0.0505344	total: 225ms	remaining: 12.3s
18:	learn: 0.0473527	total: 238ms	remaining: 12.3s
19:	learn: 0.0444205	total: 242ms	remaini

164:	learn: 0.0029428	total: 665ms	remaining: 3.37s
165:	learn: 0.0029409	total: 670ms	remaining: 3.36s
166:	learn: 0.0029147	total: 673ms	remaining: 3.36s
167:	learn: 0.0029061	total: 678ms	remaining: 3.36s
168:	learn: 0.0029061	total: 683ms	remaining: 3.36s
169:	learn: 0.0029061	total: 685ms	remaining: 3.35s
170:	learn: 0.0028697	total: 692ms	remaining: 3.35s
171:	learn: 0.0028671	total: 695ms	remaining: 3.34s
172:	learn: 0.0028648	total: 698ms	remaining: 3.33s
173:	learn: 0.0028641	total: 700ms	remaining: 3.32s
174:	learn: 0.0028436	total: 703ms	remaining: 3.31s
175:	learn: 0.0028414	total: 707ms	remaining: 3.31s
176:	learn: 0.0028184	total: 710ms	remaining: 3.3s
177:	learn: 0.0027456	total: 712ms	remaining: 3.29s
178:	learn: 0.0027133	total: 715ms	remaining: 3.28s
179:	learn: 0.0026842	total: 717ms	remaining: 3.27s
180:	learn: 0.0026783	total: 720ms	remaining: 3.26s
181:	learn: 0.0026611	total: 723ms	remaining: 3.25s
182:	learn: 0.0026309	total: 725ms	remaining: 3.24s
183:	learn: 0

327:	learn: 0.0016734	total: 1.15s	remaining: 2.35s
328:	learn: 0.0016630	total: 1.15s	remaining: 2.35s
329:	learn: 0.0016629	total: 1.16s	remaining: 2.35s
330:	learn: 0.0016590	total: 1.16s	remaining: 2.34s
331:	learn: 0.0016579	total: 1.16s	remaining: 2.34s
332:	learn: 0.0016428	total: 1.17s	remaining: 2.34s
333:	learn: 0.0016096	total: 1.17s	remaining: 2.33s
334:	learn: 0.0016096	total: 1.17s	remaining: 2.33s
335:	learn: 0.0016038	total: 1.18s	remaining: 2.32s
336:	learn: 0.0015895	total: 1.18s	remaining: 2.32s
337:	learn: 0.0015895	total: 1.18s	remaining: 2.31s
338:	learn: 0.0015722	total: 1.18s	remaining: 2.31s
339:	learn: 0.0015604	total: 1.19s	remaining: 2.3s
340:	learn: 0.0015596	total: 1.19s	remaining: 2.3s
341:	learn: 0.0015468	total: 1.19s	remaining: 2.29s
342:	learn: 0.0015462	total: 1.19s	remaining: 2.29s
343:	learn: 0.0015462	total: 1.2s	remaining: 2.28s
344:	learn: 0.0015461	total: 1.2s	remaining: 2.28s
345:	learn: 0.0015461	total: 1.2s	remaining: 2.27s
346:	learn: 0.001

506:	learn: 0.0011569	total: 1.64s	remaining: 1.59s
507:	learn: 0.0011569	total: 1.64s	remaining: 1.59s
508:	learn: 0.0011569	total: 1.65s	remaining: 1.59s
509:	learn: 0.0011569	total: 1.65s	remaining: 1.58s
510:	learn: 0.0011569	total: 1.65s	remaining: 1.58s
511:	learn: 0.0011569	total: 1.66s	remaining: 1.58s
512:	learn: 0.0011569	total: 1.66s	remaining: 1.57s
513:	learn: 0.0011569	total: 1.66s	remaining: 1.57s
514:	learn: 0.0011569	total: 1.67s	remaining: 1.57s
515:	learn: 0.0011569	total: 1.67s	remaining: 1.57s
516:	learn: 0.0011568	total: 1.67s	remaining: 1.56s
517:	learn: 0.0011568	total: 1.67s	remaining: 1.56s
518:	learn: 0.0011568	total: 1.68s	remaining: 1.55s
519:	learn: 0.0011568	total: 1.68s	remaining: 1.55s
520:	learn: 0.0011568	total: 1.68s	remaining: 1.55s
521:	learn: 0.0011568	total: 1.68s	remaining: 1.54s
522:	learn: 0.0011568	total: 1.69s	remaining: 1.54s
523:	learn: 0.0011568	total: 1.69s	remaining: 1.53s
524:	learn: 0.0011568	total: 1.69s	remaining: 1.53s
525:	learn: 

689:	learn: 0.0010273	total: 2.13s	remaining: 957ms
690:	learn: 0.0010272	total: 2.13s	remaining: 954ms
691:	learn: 0.0010272	total: 2.13s	remaining: 950ms
692:	learn: 0.0010270	total: 2.14s	remaining: 948ms
693:	learn: 0.0010269	total: 2.14s	remaining: 944ms
694:	learn: 0.0010268	total: 2.15s	remaining: 942ms
695:	learn: 0.0010267	total: 2.15s	remaining: 939ms
696:	learn: 0.0010267	total: 2.15s	remaining: 937ms
697:	learn: 0.0010267	total: 2.16s	remaining: 934ms
698:	learn: 0.0010266	total: 2.16s	remaining: 930ms
699:	learn: 0.0010204	total: 2.16s	remaining: 927ms
700:	learn: 0.0010204	total: 2.17s	remaining: 924ms
701:	learn: 0.0010204	total: 2.17s	remaining: 921ms
702:	learn: 0.0010203	total: 2.17s	remaining: 918ms
703:	learn: 0.0010203	total: 2.17s	remaining: 914ms
704:	learn: 0.0010157	total: 2.18s	remaining: 911ms
705:	learn: 0.0010157	total: 2.18s	remaining: 907ms
706:	learn: 0.0010118	total: 2.18s	remaining: 904ms
707:	learn: 0.0010118	total: 2.18s	remaining: 901ms
708:	learn: 

865:	learn: 0.0009374	total: 2.62s	remaining: 406ms
866:	learn: 0.0009374	total: 2.63s	remaining: 403ms
867:	learn: 0.0009286	total: 2.63s	remaining: 400ms
868:	learn: 0.0009207	total: 2.63s	remaining: 397ms
869:	learn: 0.0009207	total: 2.64s	remaining: 394ms
870:	learn: 0.0009207	total: 2.64s	remaining: 391ms
871:	learn: 0.0009207	total: 2.65s	remaining: 388ms
872:	learn: 0.0009207	total: 2.65s	remaining: 385ms
873:	learn: 0.0009207	total: 2.65s	remaining: 382ms
874:	learn: 0.0009207	total: 2.66s	remaining: 380ms
875:	learn: 0.0009207	total: 2.66s	remaining: 377ms
876:	learn: 0.0009207	total: 2.66s	remaining: 374ms
877:	learn: 0.0009207	total: 2.67s	remaining: 371ms
878:	learn: 0.0009207	total: 2.67s	remaining: 368ms
879:	learn: 0.0009207	total: 2.67s	remaining: 364ms
880:	learn: 0.0009133	total: 2.67s	remaining: 361ms
881:	learn: 0.0009133	total: 2.68s	remaining: 358ms
882:	learn: 0.0009133	total: 2.68s	remaining: 355ms
883:	learn: 0.0009133	total: 2.68s	remaining: 352ms
884:	learn: 

In [35]:
def print_regression_model_metrics(estimator, y_test, y_pred):
    print(estimator.score(x,y))
   

In [36]:
cbr = CatBoostRegressor(learning_rate=0.15).fit(X_train_reg, y_train_reg)
print_regression_model_metrics(cbr, y_test_reg, cbr.predict(X_test_reg))

0:	learn: 0.3173620	total: 3.09ms	remaining: 3.09s
1:	learn: 0.2854855	total: 8.68ms	remaining: 4.33s
2:	learn: 0.2623477	total: 11.6ms	remaining: 3.85s
3:	learn: 0.2370408	total: 14.1ms	remaining: 3.52s
4:	learn: 0.2184003	total: 16.5ms	remaining: 3.29s
5:	learn: 0.2017159	total: 18.6ms	remaining: 3.08s
6:	learn: 0.1857589	total: 22.8ms	remaining: 3.23s
7:	learn: 0.1713354	total: 25.7ms	remaining: 3.18s
8:	learn: 0.1608444	total: 28.4ms	remaining: 3.12s
9:	learn: 0.1531084	total: 31.1ms	remaining: 3.08s
10:	learn: 0.1432548	total: 33.4ms	remaining: 3s
11:	learn: 0.1363384	total: 35.4ms	remaining: 2.92s
12:	learn: 0.1315130	total: 37.7ms	remaining: 2.86s
13:	learn: 0.1275344	total: 40ms	remaining: 2.82s
14:	learn: 0.1218690	total: 42.1ms	remaining: 2.76s
15:	learn: 0.1178017	total: 44.1ms	remaining: 2.71s
16:	learn: 0.1145156	total: 47.7ms	remaining: 2.76s
17:	learn: 0.1116220	total: 50.2ms	remaining: 2.74s
18:	learn: 0.1098943	total: 52.5ms	remaining: 2.71s
19:	learn: 0.1086256	total:

190:	learn: 0.0489322	total: 486ms	remaining: 2.06s
191:	learn: 0.0488338	total: 490ms	remaining: 2.06s
192:	learn: 0.0485924	total: 492ms	remaining: 2.06s
193:	learn: 0.0480807	total: 495ms	remaining: 2.05s
194:	learn: 0.0479194	total: 498ms	remaining: 2.06s
195:	learn: 0.0477537	total: 500ms	remaining: 2.05s
196:	learn: 0.0475202	total: 503ms	remaining: 2.05s
197:	learn: 0.0473078	total: 506ms	remaining: 2.05s
198:	learn: 0.0472244	total: 508ms	remaining: 2.04s
199:	learn: 0.0471609	total: 513ms	remaining: 2.05s
200:	learn: 0.0470709	total: 515ms	remaining: 2.05s
201:	learn: 0.0469331	total: 518ms	remaining: 2.04s
202:	learn: 0.0467876	total: 520ms	remaining: 2.04s
203:	learn: 0.0467481	total: 522ms	remaining: 2.04s
204:	learn: 0.0462452	total: 525ms	remaining: 2.04s
205:	learn: 0.0460648	total: 528ms	remaining: 2.03s
206:	learn: 0.0460127	total: 530ms	remaining: 2.03s
207:	learn: 0.0459433	total: 532ms	remaining: 2.02s
208:	learn: 0.0458164	total: 534ms	remaining: 2.02s
209:	learn: 

389:	learn: 0.0266158	total: 977ms	remaining: 1.53s
390:	learn: 0.0265912	total: 981ms	remaining: 1.53s
391:	learn: 0.0265381	total: 983ms	remaining: 1.52s
392:	learn: 0.0264500	total: 985ms	remaining: 1.52s
393:	learn: 0.0262809	total: 988ms	remaining: 1.52s
394:	learn: 0.0261760	total: 990ms	remaining: 1.52s
395:	learn: 0.0260776	total: 993ms	remaining: 1.51s
396:	learn: 0.0260345	total: 996ms	remaining: 1.51s
397:	learn: 0.0259157	total: 998ms	remaining: 1.51s
398:	learn: 0.0258409	total: 1s	remaining: 1.51s
399:	learn: 0.0256739	total: 1s	remaining: 1.51s
400:	learn: 0.0256115	total: 1.01s	remaining: 1.5s
401:	learn: 0.0255902	total: 1.01s	remaining: 1.5s
402:	learn: 0.0255694	total: 1.01s	remaining: 1.5s
403:	learn: 0.0255073	total: 1.01s	remaining: 1.49s
404:	learn: 0.0254576	total: 1.01s	remaining: 1.49s
405:	learn: 0.0254321	total: 1.02s	remaining: 1.49s
406:	learn: 0.0254212	total: 1.02s	remaining: 1.49s
407:	learn: 0.0253913	total: 1.02s	remaining: 1.48s
408:	learn: 0.0253574

557:	learn: 0.0181080	total: 1.47s	remaining: 1.16s
558:	learn: 0.0180939	total: 1.47s	remaining: 1.16s
559:	learn: 0.0180701	total: 1.48s	remaining: 1.16s
560:	learn: 0.0180550	total: 1.49s	remaining: 1.16s
561:	learn: 0.0180191	total: 1.49s	remaining: 1.16s
562:	learn: 0.0179830	total: 1.5s	remaining: 1.16s
563:	learn: 0.0179598	total: 1.5s	remaining: 1.16s
564:	learn: 0.0179437	total: 1.51s	remaining: 1.16s
565:	learn: 0.0178636	total: 1.51s	remaining: 1.16s
566:	learn: 0.0177112	total: 1.52s	remaining: 1.16s
567:	learn: 0.0176984	total: 1.52s	remaining: 1.16s
568:	learn: 0.0176862	total: 1.53s	remaining: 1.16s
569:	learn: 0.0176164	total: 1.53s	remaining: 1.15s
570:	learn: 0.0175680	total: 1.54s	remaining: 1.15s
571:	learn: 0.0175371	total: 1.54s	remaining: 1.16s
572:	learn: 0.0175010	total: 1.55s	remaining: 1.15s
573:	learn: 0.0174228	total: 1.55s	remaining: 1.15s
574:	learn: 0.0174116	total: 1.56s	remaining: 1.15s
575:	learn: 0.0173962	total: 1.56s	remaining: 1.15s
576:	learn: 0.

724:	learn: 0.0129841	total: 1.94s	remaining: 737ms
725:	learn: 0.0129415	total: 1.95s	remaining: 734ms
726:	learn: 0.0129217	total: 1.95s	remaining: 731ms
727:	learn: 0.0129141	total: 1.95s	remaining: 729ms
728:	learn: 0.0128922	total: 1.95s	remaining: 726ms
729:	learn: 0.0128647	total: 1.96s	remaining: 724ms
730:	learn: 0.0128441	total: 1.96s	remaining: 721ms
731:	learn: 0.0128321	total: 1.96s	remaining: 718ms
732:	learn: 0.0128254	total: 1.97s	remaining: 716ms
733:	learn: 0.0128166	total: 1.97s	remaining: 713ms
734:	learn: 0.0127940	total: 1.97s	remaining: 711ms
735:	learn: 0.0127792	total: 1.97s	remaining: 708ms
736:	learn: 0.0127616	total: 1.98s	remaining: 705ms
737:	learn: 0.0127403	total: 1.98s	remaining: 702ms
738:	learn: 0.0127251	total: 1.98s	remaining: 700ms
739:	learn: 0.0127149	total: 1.98s	remaining: 697ms
740:	learn: 0.0127002	total: 1.98s	remaining: 694ms
741:	learn: 0.0126821	total: 1.99s	remaining: 691ms
742:	learn: 0.0126618	total: 1.99s	remaining: 688ms
743:	learn: 

919:	learn: 0.0096372	total: 2.43s	remaining: 211ms
920:	learn: 0.0096300	total: 2.43s	remaining: 209ms
921:	learn: 0.0096185	total: 2.44s	remaining: 206ms
922:	learn: 0.0096039	total: 2.44s	remaining: 203ms
923:	learn: 0.0095787	total: 2.44s	remaining: 201ms
924:	learn: 0.0095700	total: 2.44s	remaining: 198ms
925:	learn: 0.0095612	total: 2.45s	remaining: 195ms
926:	learn: 0.0095491	total: 2.45s	remaining: 193ms
927:	learn: 0.0095352	total: 2.45s	remaining: 190ms
928:	learn: 0.0095072	total: 2.46s	remaining: 188ms
929:	learn: 0.0095022	total: 2.46s	remaining: 185ms
930:	learn: 0.0094864	total: 2.46s	remaining: 182ms
931:	learn: 0.0094781	total: 2.46s	remaining: 180ms
932:	learn: 0.0094654	total: 2.46s	remaining: 177ms
933:	learn: 0.0094346	total: 2.47s	remaining: 174ms
934:	learn: 0.0094309	total: 2.47s	remaining: 172ms
935:	learn: 0.0094050	total: 2.47s	remaining: 169ms
936:	learn: 0.0093910	total: 2.48s	remaining: 166ms
937:	learn: 0.0093838	total: 2.48s	remaining: 164ms
938:	learn: 